### Calculating Beam-Id for all points

In [2]:
import os
import numpy as np
# CONFIGURATION
DATA_PATH = os.path.join('buni','dataset','pandaset')
SCENE_IDX = 3
FRAME_IDX = 70
GROUND_LABELS = np.array([6, 7, 8, 9, 10, 11, 12, 34, 35, 37, 38, 39])
user_home = os.path.expanduser('~') 
dataset_path = os.path.join(user_home,DATA_PATH)

In [ ]:
from pandaset import DataSet
import logging

dataset = DataSet(dataset_path)
scenes_with_semantic_labels = sorted(dataset.sequences(with_semseg=True), key=int)
print(f"List of sequences available with semantic segmentation:\n{scenes_with_semantic_labels}")
scene = dataset[scenes_with_semantic_labels[SCENE_IDX]]
print(f"Selected scene/sequence: {scenes_with_semantic_labels[SCENE_IDX]}")
scene.load_lidar()
scene.load_semseg()
logger = logging.getLogger(__name__)
logging.info(f"Loaded scene {SCENE_IDX} with frame {FRAME_IDX}")
lidar_data = scene.lidar.data[FRAME_IDX]
lidar_poses = scene.lidar.poses[FRAME_IDX]
labels = scene.semseg.data[FRAME_IDX]

from general_utils import pandaset_utils as pdutils
from general_utils import gen_utils

gen_utils.check_type(lidar_data,"lidar_data", logger)
gen_utils.check_type(labels,"labels", logger)

lidar_data, lidar_labels = pdutils.cleanup_lidar_data_and_labels(lidar_data, labels, lidar_poses,logger)

logger.info(f"Shape of lidar_data before reshaping: {lidar_data.shape}")
gen_utils.check_type(lidar_data,"lidar_data",logger)
lidar_points = lidar_data.iloc[:,:3].to_numpy()
lidar_points = lidar_points.astype('float64')
logger.info(f"Shape of lidar_data after reshaping: {lidar_points.shape}")
gen_utils.check_type(lidar_points,"lidar_points",logger)

In [5]:
import pandas as pd
import os
from general_utils import pandaset_utils as pdutils
import ray_dropping as ray_dropper

corrections_file = os.path.join(os.getcwd(),'Pandar64_Angle_Correction_File.csv')
corrections = pd.read_csv(corrections_file)

beam_ids, _, _ = pdutils.calculate_beamIds(lidar_points,corrections, True)

lidar_points_with_beam_ids = lidar_points_with_beam_ids = np.hstack((lidar_points, beam_ids.reshape(-1, 1)))

augmented_points = ray_dropper.drop_rays(lidar_points_with_beam_ids)

from general_utils import gen_utils

gen_utils.visualize_beam_ids(augmented_points[:,:3], augmented_points[:,3])